# <img align="left" src="./images/film_strip_vertical.png"     style=" width:40px;  " > ห้องปฏิบัติการฝึกหัด: การเรียนรู้เชิงลึกสำหรับการกรองตามเนื้อหา (Deep Learning for Content-Based Filtering)

ในแบบฝึกหัดนี้ คุณจะได้ฝึกปฏิบัติการกรองตามเนื้อหาโดยใช้เครือข่ายประสาทเทียมเพื่อสร้างระบบแนะนำภาพยนตร์



# บทนำ
- [ 1 - แพ็คเกจ ](#1)
- [ 2 - ชุดข้อมูลการให้คะแนนภาพยนตร์ ](#2)
- [ 3 - การกรองตามเนื้อหาด้วยเครือข่ายประสาทเทียม](#3)
  - [ 3.1 ข้อมูลฝึกสอน](#3.1)
  - [ 3.2 การเตรียมข้อมูลฝึกสอน](#3.2)
- [ 4 - เครือข่ายประสาทเทียมสำหรับการกรองตามเนื้อหา](#4)
  - [ แบบฝึกหัด 1](#ex01)
- [ 5 - การทำนาย](#5)
  - [ 5.1 - การทำนายสำหรับผู้ใช้ใหม่](#5.1)
  - [ 5.2 - การทำนายสำหรับผู้ใช้ที่มีอยู่](#5.2)
  - [ 5.3 - การค้นหาไอเท็มที่คล้ายกัน](#5.3)
    - [ แบบฝึกหัด 2](#ex02)
- [ 6 - ขอแสดงความยินดี! ](#6)


<a name="1"></a>
## 1 - แพ็คเกจ <img align="left" src="./images/movie_camera.png"     style=" width:40px;  ">
เราจะใช้แพ็คเกจที่คุ้นเคย NumPy, TensorFlow และฟังก์ชันที่มีประโยชน์จาก [scikit-learn](https://scikit-learn.org/stable/).นอกจากนี้เรายังจะใช้ [tabulate](https://pypi.org/project/tabulate/) เพื่อพิมพ์ตารางอย่างสวยงามและ  [Pandas](https://pandas.pydata.org/) เพื่อจัดระเบียบข้อมูลตาราง

In [ ]:
!pip install pickle5

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
try:
  %matplotlib widget
  print("widget is already installed")
except:
  print("widget is not been installed, install now..")
  !pip install ipympl

In [ ]:
!git clone https://github.com/Smith-WeStrideTH/Unsupervised_Learning_Course.git
%cd Unsupervised_Learning_Course/work 

In [ ]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
from recsysNN_utils_r2 import *
pd.set_option("display.precision", 1)
from google.colab import data_table
data_table.enable_dataframe_formatter()

<a name="2"></a>
## 2 - ชุดข้อมูลการให้คะแนนภาพยนตร์ <img align="left" src="./images/film_rating.png" style=" width:40px;" >
ชุดข้อมูลที่ใช้คือ [MovieLens ml-latest-small](https://grouplens.org/datasets/movielens/latest/)

[F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>]

ชุดข้อมูลเดิมมีภาพยนตร์ประมาณ 9,000 เรื่องที่ได้รับการจัดอันดับโดยผู้ใช้ 600 คน โดยมีคะแนนตั้งแต่ 0.5 ถึง 5 ในช่วงเวลา 0.5 ขั้นตอน ชุดข้อมูลนี้ได้ถูกย่อขนาดลงเพื่อมุ่งเน้นไปที่ภาพยนตร์ตั้งแต่ปี 2000 เป็นต้นไป และแนวเพลงยอดนิยม ชุดข้อมูลที่ลดขนาดลงมีผู้ใช้  $n_u = 397$ คน, ภาพยนตร์  $n_m= 847$ เรื่อง และ 25,521 การให้คะแนน สำหรับแต่ละภาพยนตร์ ชุดข้อมูลจะให้ชื่อภาพยนตร์ วันที่วางจำหน่าย และหนึ่งหรือหลายแนวเพลง ตัวอย่างเช่น "Toy Story 3" วางจำหน่ายในปี 2010 และมีหลายแนวเพลง: "Adventure|Animation|Children|Comedy|Fantasy" ชุดข้อมูลนี้มีข้อมูลเกี่ยวกับผู้ใช้น้อยมากนอกเหนือจากการให้คะแนนของพวกเขา ชุดข้อมูลนี้ถูกนำไปใช้เพื่อสร้างเวกเตอร์การฝึกอบรมสำหรับเครือข่ายประสาทเทียมที่อธิบายไว้ด้านล่าง

มาเรียนรู้เพิ่มเติมเกี่ยวกับชุดข้อมูลนี้กัน ตารางด้านล่างแสดงภาพยนตร์ 10 อันดับแรกที่จัดอันดับตามจำนวนการให้คะแนน ภาพยนตร์เหล่านี้ยังมีคะแนนเฉลี่ยสูงอีกด้วย คุณเคยดูภาพยนตร์เหล่านี้กี่เรื่อง

In [ ]:
top10_df = pd.read_csv("./data/content_top10_df.csv")
bygenre_df = pd.read_csv("./data/content_bygenre_df.csv")
top10_df

ตารางต่อไปนี้แสดงข้อมูลที่เรียงลำดับตามประเภทภาพยนตร์ จำนวนการให้คะแนนต่อประเภทมีความแตกต่างกันอย่างมาก โปรดทราบว่าภาพยนตร์หนึ่งเรื่องอาจมีหลายประเภท ดังนั้นผลรวมของการให้คะแนนด้านล่างจึงมากกว่าจำนวนการให้คะแนนดั้งเดิม

In [ ]:
bygenre_df

<a name="3"></a>
## 3 - การกรองตามเนื้อหาด้วยเครือข่ายประสาทเทียม (Content-based filtering with a neural network)

ในห้องปฏิบัติการการกรองแบบร่วมมือ คุณได้สร้างเวกเตอร์สองตัวคือ เวกเตอร์ผู้ใช้และเวกเตอร์รายการ/ภาพยนตร์ ซึ่งผลคูณดอทของทั้งสองจะทำนายคะแนน เวกเตอร์เหล่านี้ได้มาจากการให้คะแนนเท่านั้น

การกรองตามเนื้อหาก็สร้างเวกเตอร์คุณลักษณะของผู้ใช้และภาพยนตร์เช่นกัน แต่ตระหนักว่าอาจมีข้อมูลเพิ่มเติมเกี่ยวกับผู้ใช้และ/หรือภาพยนตร์ที่อาจปรับปรุงการคาดการณ์ ข้อมูลเพิ่มเติมถูกนำไปใช้กับเครือข่ายประสาทเทียมซึ่งจะสร้างเวกเตอร์ผู้ใช้และภาพยนตร์ตามที่แสดงด้านล่าง


<figure>
    <center> <img src="./images/RecSysNN.png"   style="width:500px;height:280px;" ></center>
</figure>

<a name="3.1"></a>
### 3.1 ชุดข้อมูลฝึก (Training Data)
เนื้อหาของภาพยนตร์ที่จัดเตรียมให้กับเครือข่ายเป็นการผสมผสานระหว่างข้อมูลต้นฉบับและ 'ฟีเจอร์ที่ถูกออกแบบ' ระลึกถึงการอภิปรายเกี่ยวกับการออกแบบฟีเจอร์และห้องปฏิบัติการจากหลักสูตร 1 สัปดาห์ที่ 2 `C2_Lab04_FeatEng_PolyReg` ฟีเจอร์ดั้งเดิมคือปีที่ภาพยนตร์ถูกปล่อยและประเภทของภาพยนตร์ที่นำเสนอเป็นเวกเตอร์แบบ one-hot มี 14 ประเภท ฟีเจอร์ที่ออกแบบคือค่าเฉลี่ยของการให้คะแนนที่ได้จากการให้คะแนนของผู้ใช้

เนื้อหาของผู้ใช้ประกอบด้วยฟีเจอร์ที่ออกแบบ คำนวณค่าเฉลี่ยการให้คะแนนต่อประเภทต่อผู้ใช้ นอกจากนี้ ยังมีรหัสผู้ใช้ จำนวนการให้คะแนน และค่าเฉลี่ยการให้คะแนน แต่ไม่ได้รวมอยู่ในเนื้อหาการฝึกหรือการทำนาย ข้อมูลเหล่านี้ถูกนำไปพร้อมกับชุดข้อมูลเนื่องจากมีประโยชน์ในการตีความข้อมูล

ชุดข้อมูลการฝึกประกอบด้วยการให้คะแนนทั้งหมดที่ผู้ใช้ทำในชุดข้อมูล การให้คะแนนบางอย่างถูกทำซ้ำเพื่อเพิ่มจำนวนตัวอย่างการฝึกของประเภทที่แสดงไม่เพียงพอ ชุดฝึกอบรมถูกแบ่งออกเป็นสองอาร์เรย์ที่มีจำนวนรายการเท่ากัน อาร์เรย์ผู้ใช้และอาร์เรย์ภาพยนตร์/รายการ

ด้านล่างนี้ มาโหลดและแสดงข้อมูลบางส่วนกัน

In [ ]:
# Load Data, set configuration variables
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()

num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items
print(f"Number of training vectors: {len(item_train)}")

มาดูรายการแรก ๆ ในอาร์เรย์การฝึกอบรมผู้ใช้กัน

In [ ]:
pprint_train(user_train, user_features, uvs,  u_s, maxcount=5)

บางส่วนของคุณลักษณะผู้ใช้และรายการ/ภาพยนตร์ไม่ได้ถูกนำมาใช้ในการฝึกอบรม ในตารางด้านบน คุณลักษณะในวงเล็บ "[ ]" เช่น "รหัสผู้ใช้" "จำนวนการให้คะแนน" และ "ค่าเฉลี่ยการให้คะแนน" ไม่รวมอยู่เมื่อฝึกอบรมและใช้โมเดล ด้านบน คุณสามารถดูค่าเฉลี่ยการให้คะแนนต่อประเภทของผู้ใช้ที่ 2 รายการที่มีค่าเป็นศูนย์คือประเภทที่ผู้ใช้ยังไม่ได้ให้คะแนน เวกเตอร์ผู้ใช้เหมือนกันสำหรับภาพยนตร์ทั้งหมดที่ผู้ใช้ให้คะแนน  มาดูรายการแรกๆ ของอาร์เรย์ภาพยนตร์/รายการกัน








In [ ]:
pprint_train(item_train, item_features, ivs, i_s, maxcount=5, user=False)

ด้านบน อาร์เรย์ภาพยนตร์ประกอบด้วยปีที่ภาพยนตร์ออกฉาย คะแนนเฉลี่ย และตัวบ่งชี้สำหรับแต่ละประเภทที่เป็นไปได้ ตัวบ่งชี้คือหนึ่งสำหรับแต่ละประเภทที่ใช้กับภาพยนตร์ รหัสภาพยนตร์ไม่ได้ใช้ในการฝึกอบรม แต่มีประโยชน์เมื่อตีความข้อมูล

In [ ]:
print(f"y_train[:5]: {y_train[:5]}")

เป้าหมาย y คือคะแนนภาพยนตร์ที่ผู้ใช้ให้ไว้

จากด้านบน เราสามารถเห็นได้ว่าภาพยนตร์ 6874 เป็นภาพยนตร์แอ็คชั่น/อาชญากรรม/ระทึกขวัญที่ออกฉายในปี 2003 ผู้ใช้หมายเลข 2 ให้คะแนนภาพยนตร์แอ็คชั่นโดยเฉลี่ย 3.9 ผู้ใช้ MovieLens ให้คะแนนภาพยนตร์โดยเฉลี่ย 4 'y' คือ 4 ซึ่งบ่งชี้ว่าผู้ใช้หมายเลข 2 ให้คะแนนภาพยนตร์ 6874 เป็น 4 เช่นกัน ข้อมูลตัวอย่างการฝึกอบรมหนึ่งชุดประกอบด้วยแถวหนึ่งจากทั้งอาร์เรย์ผู้ใช้และอาร์เรย์รายการ และคะแนนจาก y_train

<a name="3.2"></a>
### 3.2 การเตรียมข้อมูลฝึกอบรม
นึกย้อนกลับไปในหลักสูตรที่ 1 สัปดาห์ที่ 2 (C2) คุณได้สำรวจการปรับขนาดคุณลักษณะ (feature scaling) เป็นวิธีการหนึ่งในการปรับปรุงการลู่เข้า เราจะปรับขนาดคุณลักษณะอินพุตโดยใช้ [scikit learn StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). สิ่งนี้ถูกใช้ใน `C2_Lab03_Feature_Scaling_and_Learning_Rate` ด้านล่าง inverse_transform ยังแสดงเพื่อสร้างอินพุตต้นฉบับ เราจะปรับขนาดคะแนนเป้าหมายโดยใช้ Min Max Scaler ซึ่งปรับขนาดเป้าหมายให้อยู่ระหว่าง -1 ถึง 1 [scikit learn MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)

In [ ]:
# scale training data
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

เพื่อให้เราสามารถประเมินผลลัพธ์ได้ เราจะแบ่งข้อมูลออกเป็นชุดฝึกอบรมและชุดทดสอบ ตามที่ได้กล่าวไว้ในหลักสูตร 2 สัปดาห์ที่ 3 (c6) `C6_Lab07_Model_Evaluation_and_Selection` ที่นี่เราจะใช้ [sklean train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) เพื่อแบ่งและสุ่มข้อมูล โปรดทราบว่าการตั้งค่าสถานะแบบสุ่มเริ่มต้นเป็นค่าเดียวกันจะทำให้มั่นใจได้ว่าไอเท็ม ผู้ใช้ และ y ถูกสุ่มเหมือนกัน

In [ ]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

ข้อมูลที่ปรับขนาดและสุ่มแล้วมีค่าเฉลี่ยเป็นศูนย์

In [ ]:
pprint_train(user_train, user_features, uvs, u_s, maxcount=5)

<a name="4"></a>
## 4 - เครือข่ายประสาทเทียมสำหรับการกรองตามเนื้อหา
ตอนนี้เรามาสร้างเครือข่ายประสาทเทียมตามที่อธิบายไว้ในรูปด้านบน เครือข่ายนี้จะมีสองเครือข่ายที่เชื่อมกันด้วยดอทพรักต์ คุณจะสร้างเครือข่ายทั้งสองนี้ ในตัวอย่างนี้ เครือข่ายทั้งสองจะเป็นเหมือนกัน โปรดทราบว่าเครือข่ายเหล่านี้ไม่จำเป็นต้องเหมือนกันเสมอไป. หากเนื้อหาของผู้ใช้มีขนาดใหญ่กว่าเนื้อหาของภาพยนตร์ คุณอาจเลือกที่จะเพิ่มความซับซ้อนของเครือข่ายผู้ใช้เมื่อเทียบกับเครือข่ายภาพยนตร์ ในกรณีนี้ เนื้อหาคล้ายกัน ดังนั้นเครือข่ายจึงเหมือนกัน

<a name="ex01"></a>
### แบบฝึกหัดที่ 1
- ใช้โมเดล sequential ของ Keras
    - เลเยอร์แรกเป็นเลเยอร์แบบ dense ที่มี 256 ยูนิตและฟังก์ชันการกระตุ้น relu
    - เลเยอร์ที่สองเป็นเลเยอร์แบบ dense ที่มี 128 ยูนิตและฟังก์ชันการกระตุ้น relu
    - เลเยอร์ที่สามเป็นเลเยอร์แบบ dense ที่มี  `num_outputs` ยูนิตและฟังก์ชันการกระตุ้นแบบ linear หรือไม่มีฟังก์ชันการกระตุ้น
    
ส่วนที่เหลือของเครือข่ายจะถูกกำหนด โค้ดที่ให้มานี้ไม่ได้ใช้โมเดล sequential ของ Keras แต่ใช้ Keras [functional api](https://keras.io/guides/functional_api/). ฟอร์แมตนี้มีความยืดหยุ่นมากกว่าในการเชื่อมต่อองค์ประกอบต่างๆ


In [ ]:
# GRADED_CELL
# UNQ_C1

num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  
  
  
    ### END CODE HERE ###  
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  
  
  
    ### END CODE HERE ###  
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

In [ ]:
# Public tests
from public_tests_r2 import *
test_tower(user_NN)
test_tower(item_NN)

<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    
คุณสามารถสร้างชั้นหนาแน่น (Dense Layer) ที่มีฟังก์ชันการกระตุ้น ReLU ดังที่แสดง

```python     
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  tf.keras.layers.Dense(256, activation='relu'),

    
    ### END CODE HERE ###  
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  tf.keras.layers.Dense(256, activation='relu'),

    
    ### END CODE HERE ###  
])
```    
<details>
    <summary><font size="2" color="darkblue"><b> ดูวิธีการทำ </b></font></summary>
    
```python 
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_outputs),
    ### END CODE HERE ###  
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_outputs),
    ### END CODE HERE ###  
])
```
</details>
</details>

    


เราจะใช้ฟังก์ชันการสูญเสียแบบ Mean Squared Error และตัวเพิ่มประสิทธิภาพ Adam

In [ ]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [ ]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

ประเมินโมเดลเพื่อกำหนดค่า loss บนชุดข้อมูลทดสอบ

In [ ]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

การสูญเสียในการฝึกอบรมนั้นเทียบเท่ากับการบ่งชี้ว่าโมเดลไม่ได้โอเวอร์ฟิตข้อมูลการฝึกมากเกินไป

<a name="5"></a>
## 5 - การทำนาย
ด้านล่าง คุณจะใช้แบบจำลองของคุณเพื่อทำนายในหลายสถานการณ์

<a name="5.1"></a>
### 5.1 - การคาดการณ์สำหรับผู้ใช้ใหม่
อันดับแรก เราจะสร้างผู้ใช้ใหม่และให้โมเดลแนะนำภาพยนตร์สำหรับผู้ใช้รายนั้น หลังจากที่คุณลองใช้เนื้อหาตัวอย่างของผู้ใช้แล้ว คุณสามารถเปลี่ยนเนื้อหาผู้ใช้ให้ตรงกับความชอบของคุณเองและดูว่าโมเดลแนะนำอะไร โปรดทราบว่าการให้คะแนนอยู่ระหว่าง 0.5 ถึง 5.0 รวมทั้งการเพิ่มทีละครึ่งก้าว

In [ ]:
new_user_id = 5000
new_rating_ave = 0.0
new_action = 0.0
new_adventure = 5.0
new_animation = 0.0
new_childrens = 0.0
new_comedy = 0.0
new_crime = 0.0
new_documentary = 0.0
new_drama = 0.0
new_fantasy = 5.0
new_horror = 0.0
new_mystery = 0.0
new_romance = 0.0
new_scifi = 0.0
new_thriller = 0.0
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

ผู้ใช้ใหม่ชอบภาพยนตร์แนวผจญภัยและแฟนตาซี มาหาภาพยนตร์ยอดนิยมสำหรับผู้ใช้ใหม่กัน
ด้านล่างนี้ เราจะใช้ชุดเวกเตอร์ภาพยนตร์/รายการ `item_vecs` ซึ่งมีเวกเตอร์สำหรับแต่ละภาพยนตร์ในชุดฝึกสอน/ทดสอบ นี่จะจับคู่กับเวกเตอร์ผู้ใช้ใหม่ด้านบน และเวกเตอร์ที่ปรับขนาดแล้วจะถูกนำมาใช้เพื่อคาดคะเนคะแนนของภาพยนตร์ทั้งหมด"

In [ ]:
# generate and replicate the user vector to match the number movies in the data set.
user_vecs = gen_user_vecs(user_vec,len(item_vecs))

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display

print_pred_movies(sorted_ypu, sorted_items, movie_dict, maxcount = 10)

<a name="5.2"></a>
### 5.2 - การคาดการณ์สำหรับผู้ใช้ที่มีอยู่
มาดูการคาดการณ์สำหรับ "ผู้ใช้ 2" หนึ่งในผู้ใช้ในชุดข้อมูลกัน เราสามารถเปรียบเทียบการให้คะแนนที่คาดการณ์กับการให้คะแนนของโมเดล

In [ ]:
uid = 2 
# form a set of user vectors. This is the same vector, transformed and repeated.
user_vecs, y_vecs = get_user_vecs(uid, user_train_unscaled, item_vecs, user_to_genre)

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display
sorted_user  = user_vecs[sorted_index]
sorted_y     = y_vecs[sorted_index]

#print sorted predictions for movies rated by the user
print_existing_user(sorted_ypu, sorted_y.reshape(-1,1), sorted_user, sorted_items, ivs, uvs, movie_dict, maxcount = 50)

โดยทั่วไปแล้ว การทำนายของโมเดลจะอยู่ภายในค่าเบี่ยงเบน 1 คะแนนจากคะแนนจริง แม้ว่าจะไม่ใช่ตัวทำนายที่แม่นยำมากนักว่าผู้ใช้จะให้คะแนนภาพยนตร์เฉพาะเรื่องอย่างไร โดยเฉพาะอย่างยิ่งหากคะแนนของผู้ใช้นั้นแตกต่างจากค่าเฉลี่ยประเภทภาพยนตร์ของผู้ใช้อย่างมาก คุณสามารถเปลี่ยน user id ด้านบนเพื่อลองผู้ใช้คนอื่นได้ ไม่ใช่ทุก user id ที่ถูกนำมาใช้ในการฝึกโมเดล

<a name="5.3"></a>
### 5.3 - การค้นหาไอเท็มที่คล้ายกัน

เครือข่ายประสาทเทียมข้างต้นสร้างเวกเตอร์คุณลักษณะสองตัว คือ เวกเตอร์คุณลักษณะผู้ใช้  $v_u$, และเวกเตอร์คุณลักษณะภาพยนตร์ $v_m$. ซึ่งเป็นเวกเตอร์ 32 รายการที่มีค่าที่ยากต่อการตีความ อย่างไรก็ตาม ไอเท็มที่คล้ายกันจะมีเวกเตอร์ที่คล้ายกัน ข้อมูลนี้สามารถนำไปใช้ในการแนะนำ ตัวอย่างเช่น หากผู้ใช้ให้คะแนน "Toy Story 3" สูง คุณสามารถแนะนำภาพยนตร์ที่คล้ายกันโดยเลือกภาพยนตร์ที่มีเวกเตอร์คุณลักษณะภาพยนตร์คล้ายกัน

มาตรการความคล้ายคลึงกัน (A similarity measure) คือระยะห่างกำลังสองระหว่างเวกเตอร์ทั้งสอง $ \mathbf{v_m^{(k)}}$ and $\mathbf{v_m^{(i)}}$ :
$$\left\Vert \mathbf{v_m^{(k)}} - \mathbf{v_m^{(i)}}  \right\Vert^2 = \sum_{l=1}^{n}(v_{m_l}^{(k)} - v_{m_l}^{(i)})^2\tag{1}$$

<a name="ex02"></a>
### แบบฝึกหัดที่ 2

เขียนฟังก์ชันเพื่อคำนวณระยะทางกำลังสอง

In [ ]:
# GRADED_FUNCTION: sq_dist
# UNQ_C2
def sq_dist(a,b):
    """
    Returns the squared distance between two vectors
    Args:
      a (ndarray (n,)): vector with n features
      b (ndarray (n,)): vector with n features
    Returns:
      d (float) : distance
    """
    ### START CODE HERE ###     
    
    ### END CODE HERE ###     
    return d

In [ ]:
a1 = np.array([1.0, 2.0, 3.0]); b1 = np.array([1.0, 2.0, 3.0])
a2 = np.array([1.1, 2.1, 3.1]); b2 = np.array([1.0, 2.0, 3.0])
a3 = np.array([0, 1, 0]);       b3 = np.array([1, 0, 0])
print(f"squared distance between a1 and b1: {sq_dist(a1, b1):0.3f}")
print(f"squared distance between a2 and b2: {sq_dist(a2, b2):0.3f}")
print(f"squared distance between a3 and b3: {sq_dist(a3, b3):0.3f}")

**ผลลัพธ์คาดหวัง**:

squared distance between a1 and b1: 0.000    
squared distance between a2 and b2: 0.030   
squared distance between a3 and b3: 2.000

In [ ]:
# Public tests
test_sq_dist(sq_dist)

<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    
แม้ว่าการหาผลรวมมักบ่งบอกว่าควรใช้ลูป for แต่ที่นี่การลบสามารถทำได้ทีละองค์ประกอบในคำสั่งเดียว นอกจากนี้ คุณยังสามารถใช้ np.square เพื่อยกกำลังสอง ผลลัพธ์ของการลบทีละองค์ประกอบ np.sum สามารถใช้ในการรวมองค์ประกอบที่ยกกำลังสอง


</details>

    


เมทริกซ์ระยะห่างระหว่างภาพยนตร์สามารถคำนวณได้ครั้งเดียวเมื่อโมเดลได้รับการฝึกฝนแล้วและนำมาใช้ใหม่สำหรับคำแนะนำใหม่โดยไม่ต้องฝึกโมเดลใหม่ ขั้นตอนแรกหลังจากฝึกโมเดลเสร็จแล้วคือการรับเวกเตอร์ฟีเจอร์ของภาพยนตร์ $v_m$, สำหรับแต่ละภาพยนตร์ เพื่อทำเช่นนี้ เราจะใช้ `item_NN` ที่ได้รับการฝึกฝนและสร้างโมเดลขนาดเล็กเพื่อให้เราสามารถรันเวกเตอร์ภาพยนตร์ผ่านมันเพื่อสร้าง  $v_m$.

In [ ]:
input_item_m = tf.keras.layers.Input(shape=(num_item_features))    # input layer
vm_m = item_NN(input_item_m)                                       # use the trained item_NN
vm_m = tf.linalg.l2_normalize(vm_m, axis=1)                        # incorporate normalization as was done in the original model
model_m = tf.keras.Model(input_item_m, vm_m)                                
model_m.summary()

เมื่อคุณมีโมเดลภาพยนตร์แล้ว คุณสามารถสร้างชุดเวกเตอร์คุณลักษณะภาพยนตร์โดยใช้โมเดลเพื่อทำนายโดยใช้ชุดเวกเตอร์รายการ/ภาพยนตร์เป็นอินพุต `item_vecs`  เป็นชุดของเวกเตอร์ภาพยนตร์ทั้งหมด ต้องปรับขนาดเพื่อใช้กับโมเดลที่ได้รับการฝึกอบรม ผลลัพธ์ของการทำนายคือเวกเตอร์คุณลักษณะ 32 รายการสำหรับแต่ละภาพยนตร์

In [ ]:
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:,i_s:])
print(f"size of all predicted movie feature vectors: {vms.shape}")

ตอนนี้มาคำนวณเมทริกซ์ของระยะห่างกำลังสองระหว่างเวกเตอร์ฟีเจอร์ของภาพยนตร์แต่ละเรื่องกับเวกเตอร์ฟีเจอร์ภาพยนตร์อื่นๆ ทั้งหมดกัน
<figure>
    <left> <img src="./images/distmatrix.PNG"   style="width:400px;height:225px;" ></center>
</figure>

จากนั้นเราสามารถหาหนังที่ใกล้เคียงที่สุดได้โดยการหาค่าต่ำสุดในแต่ละแถว เราจะใช้ [numpy masked arrays](https://numpy.org/doc/1.21/user/tutorial-ma.html) ของ NumPy เพื่อหลีกเลี่ยงการเลือกหนังเรื่องเดียวกัน ค่าที่ถูกปิดบัง (masked) ตามแนวทแยงจะไม่ถูกนำไปคำนวณด้วย

In [ ]:
count = 50  # number of movies to display
dim = len(vms)
dist = np.zeros((dim,dim))

for i in range(dim):
    for j in range(dim):
        dist[i,j] = sq_dist(vms[i, :], vms[j, :])
        
m_dist = ma.masked_array(dist, mask=np.identity(dist.shape[0]))  # mask the diagonal

disp = [["movie1", "genres", "movie2", "genres"]]
for i in range(count):
    min_idx = np.argmin(m_dist[i])
    movie1_id = int(item_vecs[i,0])
    movie2_id = int(item_vecs[min_idx,0])
    disp.append( [movie_dict[movie1_id]['title'], movie_dict[movie1_id]['genres'],
                  movie_dict[movie2_id]['title'], movie_dict[movie1_id]['genres']]
               )
table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
table

ผลลัพธ์แสดงให้เห็นว่าโมเดลจะแนะนำภาพยนตร์ที่มีแนวเดียวกันโดยทั่วไป

<a name="6"></a>
## 6 - ยินดีด้วย! <img align="left" src="./images/film_award.png" style=" width:40px;">
คุณได้สร้างระบบการแนะนำเนื้อหาแบบ Content-Based Recommender System แล้ว

โครงสร้างนี้เป็นพื้นฐานของระบบการแนะนำเชิงพาณิชย์หลายระบบ เนื้อหาของผู้ใช้สามารถขยายได้อย่างมากเพื่อรวมข้อมูลเพิ่มเติมเกี่ยวกับผู้ใช้หากมีข้อมูลดังกล่าว ไอเท็มไม่จำกัดเฉพาะภาพยนตร์ เทคนิคนี้สามารถใช้เพื่อแนะนำไอเท็มใด ๆ เช่น หนังสือ รถยนต์ หรือไอเท็มที่คล้ายกับไอเท็มใน 'ตะกร้าสินค้า' ของคุณ